# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 32,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": 100,
                "recon_loss_weight": 10000,
                "ll_weight": .0001,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 10,
                "save_step": 10,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                'tb_compare': [('vaegan_1024_filter_discr', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-20_09-39-05/')],
                'no_gan': False,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNI

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-22_16-01__root.log
Starting train_vaegan main
Numpy random seed: 989304065
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 222.00
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/dense_1/BiasAdd:0: (?, 100)
encoder/dense_2/BiasAdd:0: (?, 100)
epsilon: (?, 100)
z: (?, 100)
input_z: (?, 100)
decoder/dec_dense1/BiasAdd

DEBUG_OP "dis_loss: d_x_real": 
0.5706430673599243
DEBUG_OP "dis_loss: d_x_vae": 
0.6640563011169434
DEBUG_OP "dis_loss: d_x_noise": 
0.6662873029708862
DEBUG_OP "dis_loss: 1 - d_x_vae": 
0.33594369888305664
DEBUG_OP "dis_loss: 1 - d_x_noise": 
0.333712637424469
DEBUG_OP "dis_loss": 
0.19802667200565338
Enc Loss = 4.15, KL Divergence = 0.04, Reconstruction Loss = 0.13, ll_loss = 23.01, dis_Loss = 0.20, dec_Loss = 0.20, Elapsed time: 1.83 mins
Memory Use (GB): 1.9342002868652344
Epoch: 0 / 201, Batch: 10 (0 / 352), Elapsed time: 1.83 mins
DEBUG_OP "dis_loss: d_x_real": 
0.646470308303833
DEBUG_OP "dis_loss: d_x_vae": 
0.8419999480247498
DEBUG_OP "dis_loss: d_x_noise": 
0.841303288936615
DEBUG_OP "dis_loss: 1 - d_x_vae": 
0.15800005197525024
DEBUG_OP "dis_loss: 1 - d_x_noise": 
0.15869668126106262
DEBUG_OP "dis_loss": 
0.6595472097396851
Enc Loss = 3.63, KL Divergence = 0.04, Reconstruction Loss = 0.11, ll_loss = 118.20, dis_Loss = 0.66, dec_Loss = 0.67, Elapsed time: 2.01 mins
Memory Us

DEBUG_OP "dis_loss: d_x_vae": 
0.7117142081260681
DEBUG_OP "dis_loss: d_x_noise": 
0.7106537222862244
DEBUG_OP "dis_loss: 1 - d_x_vae": 
0.2882857918739319
DEBUG_OP "dis_loss: 1 - d_x_noise": 
0.28934621810913086
DEBUG_OP "dis_loss": 
0.04176659882068634
Enc Loss = 1.98, KL Divergence = 0.02, Reconstruction Loss = 0.10, ll_loss = 53.96, dis_Loss = 0.04, dec_Loss = 0.05, Elapsed time: 4.53 mins
Memory Use (GB): 1.9432907104492188
Epoch: 0 / 201, Batch: 25 (0 / 832), Elapsed time: 4.53 mins
DEBUG_OP "dis_loss: d_x_real": 
0.5951475501060486
DEBUG_OP "dis_loss: d_x_vae": 
0.7892378568649292
DEBUG_OP "dis_loss: d_x_noise": 
0.790790319442749
DEBUG_OP "dis_loss: 1 - d_x_vae": 
0.2107621729373932
DEBUG_OP "dis_loss: 1 - d_x_noise": 
0.20920968055725098
DEBUG_OP "dis_loss": 
0.35035139322280884
Enc Loss = 1.94, KL Divergence = 0.02, Reconstruction Loss = 0.11, ll_loss = 105.39, dis_Loss = 0.35, dec_Loss = 0.36, Elapsed time: 4.71 mins
Memory Use (GB): 2.2169265747070312
Epoch: 0 / 201, Batch:

DEBUG_OP "dis_loss: d_x_noise": 
0.6875401735305786
DEBUG_OP "dis_loss: 1 - d_x_vae": 
0.30869603157043457
DEBUG_OP "dis_loss: 1 - d_x_noise": 
0.31245988607406616
DEBUG_OP "dis_loss": 
0.12026571482419968
Enc Loss = 1.47, KL Divergence = 0.01, Reconstruction Loss = 0.12, ll_loss = 49.94, dis_Loss = 0.12, dec_Loss = 0.13, Elapsed time: 7.23 mins
Memory Use (GB): 2.1238937377929688
Epoch: 0 / 201, Batch: 40 (0 / 1312), Elapsed time: 7.23 mins
